In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import re
import dateutil.parser

In [2]:
def get_movie_value(soup, field_name):
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(field_name))
    if not obj: 
        return None
    # this works for most of the values
    next_element = obj.findNext()
    if next_element:
        return next_element.text 
    else:
        return None

In [27]:
def money_to_int(moneystring):
    #if type(moneystring) = 'NoneType':
    #    moneystring = 0
    #moneystring = moneystring.replace('$', '').replace(',', '')
    #return int(moneystring)
    try:
        moneystring = moneystring.replace('$', '').replace(',', '')
        return int(moneystring)
    except :
        pass

def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

In [87]:
def movie_info(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    title_string = soup.find('title').text
    title_string.split('-')
    title = title_string.split('-')[0].strip()
    
    dtg = soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[0].text
    
    runtime = get_movie_value(soup,'Run')
    
    rating = get_movie_value(soup,'MPAA')
    
    release_date = get_movie_value(soup,'Release Date')
    release_date = release_date.split('\n')[0]
    
    distributor = get_movie_value(soup,'Domestic Distributor')
    
    budget = get_movie_value(soup,'Budget')
    budget = money_to_int(raw_budget)
    
    raw_domestic_total_gross = dtg
    domestic_total_gross = money_to_int(raw_domestic_total_gross)

    raw_runtime = get_movie_value(soup,'Running')
    runtime = runtime_to_minutes(raw_runtime)

    raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
    release_date = to_date(raw_release_date)
    
    
    
    genre = get_movie_value(soup,'Genres')
    res = [] 
    for sub in genre: 
        res.append(re.sub('\n', '', sub)) 
    res = ''.join(res)
    res = re.sub(' +', ', ', res)
    
    
    headers = ['Movie_Title', 'Domestic_Total_Gross','Budget','Distributor',
               'Runtime(mins)', 'Rating', 'Release_Date','Genre']

    movie_data = []
    movie_dict = dict(zip(headers, [title,
                                    distributor,
                                    domestic_total_gross,
                                    budget,
                                    runtime,
                                    rating, 
                                    release_date,
                                    res]))

    movie_data.append(movie_dict)
    return movie_data
    
    
    
    

In [5]:
url = 'https://www.boxofficemojo.com/release/rl2691925505/?ref_=bo_wly_table_1'
movie_info(url)

[{'Movie_Title': 'Star Wars: Episode VII',
  'Domestic_Total_Gross': 936662225,
  'Budget': 245000000,
  'Runtime(mins)': 138,
  'Rating': 'PG-13',
  'Release_Date': datetime.datetime(2015, 12, 18, 0, 0),
  'Genre': 'Action, Adventure, Sci-Fi'}]

In [6]:

#k = movie_info(url)
#item = list(k[0].values())[5]
#list(k[0].values())[5]
#res = [] 
#for sub in item: 
#    res.append(re.sub('\n', '', sub)) 

#res = ''.join(res)
#re.sub(' +', ' ', res)

In [7]:
test = 'https://www.boxofficemojo.com/weekly/by-year/2019/'
response = requests.get(test)
page = response.text
soup = BeautifulSoup(page, "lxml")
soup.text[:1000]

'var aPageStart = (new Date()).getTime();\n\nDomestic Box Office Weeklies For 2019 - Box Office Mojo\n\n\n\n\n\n\n(function(f,h,Q,A){function G(a){v&&v.tag&&v.tag(q(":","aui",a))}function x(a,b){v&&v.count&&v.count("aui:"+a,0===b?0:b||(v.count("aui:"+a)||0)+1)}function l(a){try{return a.test(navigator.userAgent)}catch(b){return!1}}function y(a,b,c){a.addEventListener?a.addEventListener(b,c,!1):a.attachEvent&&a.attachEvent("on"+b,c)}function q(a,b,c,e){b=b&&c?b+a+c:b||c;return e?q(a,b,e):b}function H(a,b,c){try{Object.defineProperty(a,b,{value:c,writable:!1})}catch(e){a[b]=c}return c}function ta(a,b){var c=a.length,\ne=c,g=function(){e--||(R.push(b),S||(setTimeout(T,0),S=!0))};for(g();c--;)ca[a[c]]?g():(B[a[c]]=B[a[c]]||[]).push(g)}function ua(a,b,c,e,g){var d=h.createElement(a?"script":"link");y(d,"error",e);g&&y(d,"load",g);a?(d.type="text/javascript",d.async=!0,c&&/AUIClients|images[/]I/.test(b)&&d.setAttribute("crossorigin","anonymous"),d.src=b):(d.rel="stylesheet",d.href=b);h.getEl

In [8]:
k = soup.find_all('a')
list(k)[27:]

[<a class="a-link-normal a-nowrap" href="?sort=weekNum&amp;ref_=bo_wly__resort#table" title="Week Number">Week<span class="a-letter-space"></span><span class="icon aok-relative"><i class="a-icon a-icon-expand table-sort-desc-placeholder" role="img"></i><i class="a-icon a-icon-collapse table-sort-asc-placeholder" role="img"></i></span></a>,
 <a class="a-link-normal" href="/weekly/2019W52/?ref_=bo_wly_table_1">Dec 27-Jan 2, 2020</a>,
 <a class="a-link-normal" href="/release/rl3305145857/?ref_=bo_wly_table_1">Star Wars: The Rise Of Skywalker</a>,
 <a class="a-link-normal" href="/weekly/2019W52/?ref_=bo_wly_table_1">52</a>,
 <a class="a-link-normal" href="/weekly/2019W51/?ref_=bo_wly_table_2">Dec 20-26</a>,
 <a class="a-link-normal" href="/release/rl3305145857/?ref_=bo_wly_table_2">Star Wars: The Rise Of Skywalker</a>,
 <a class="a-link-normal" href="/weekly/2019W51/?ref_=bo_wly_table_2">51</a>,
 <a class="a-link-normal" href="/weekly/2019W50/?ref_=bo_wly_table_3">Dec 13-19</a>,
 <a class=

In [9]:
df = pd.DataFrame(np.array(list(k)[27:-12]).reshape(-1,3))
moviepaths = df.loc[:,2]
moviepaths[0]

<a class="a-link-normal" href="/release/rl3305145857/?ref_=bo_wly_table_1">Star Wars: The Rise Of Skywalker</a>

In [10]:
moviepaths.head()

0    [Star Wars: The Rise Of Skywalker]
1    [Star Wars: The Rise Of Skywalker]
2             [Jumanji: The Next Level]
3                           [Frozen II]
4                           [Frozen II]
Name: 2, dtype: object

In [11]:
type(moviepaths)

pandas.core.series.Series

In [12]:
urls = [] 
for a_tag in moviepaths:
    url = a_tag['href']
    if not url.startswith('http'):
        url = "https://www.boxofficemojo.com/"+url
    urls.append(url)

In [13]:
urls[0:5]


['https://www.boxofficemojo.com//release/rl3305145857/?ref_=bo_wly_table_1',
 'https://www.boxofficemojo.com//release/rl3305145857/?ref_=bo_wly_table_2',
 'https://www.boxofficemojo.com//release/rl755467777/?ref_=bo_wly_table_3',
 'https://www.boxofficemojo.com//release/rl2424210945/?ref_=bo_wly_table_4',
 'https://www.boxofficemojo.com//release/rl2424210945/?ref_=bo_wly_table_5']

In [29]:
info = pd.DataFrame(columns=['Movie_Title', 'Domestic_Total_Gross', 'Runtime(mins)','Rating','Release_Date','Genre'])
for i in range(5):
    info.append(movie_info(urls[i]))

In [ ]:
#movie_info(urls[5])

In [30]:
t = movie_info(urls[10])
#print(t[0].values)

In [31]:
def getList(dict): 
      
    return [*dict] 
Header = getList(t[0])
Header

['Movie_Title',
 'Domestic_Total_Gross',
 'Budget',
 'Runtime(mins)',
 'Rating',
 'Release_Date',
 'Genre']

In [ ]:
#Values = list(t[0].values())
#Values

In [ ]:
#dft = pd.DataFrame(columns=Header)
#dft.append(pd.Series(Values, index=Header), ignore_index=True)


In [ ]:
#soup.find_all(class_='a-link-normal').find_all(href = /release*)

In [32]:
print(urls[0:5])
print(movie_info(urls[1]))
print(list(t[0].values()))
Header = getList(t[0])

['https://www.boxofficemojo.com//release/rl3305145857/?ref_=bo_wly_table_1', 'https://www.boxofficemojo.com//release/rl3305145857/?ref_=bo_wly_table_2', 'https://www.boxofficemojo.com//release/rl755467777/?ref_=bo_wly_table_3', 'https://www.boxofficemojo.com//release/rl2424210945/?ref_=bo_wly_table_4', 'https://www.boxofficemojo.com//release/rl2424210945/?ref_=bo_wly_table_5']
[{'Movie_Title': 'Star Wars: The Rise Of Skywalker', 'Domestic_Total_Gross': 482569280, 'Budget': None, 'Runtime(mins)': 142, 'Rating': 'PG-13', 'Release_Date': datetime.datetime(2019, 12, 20, 0, 0), 'Genre': 'Action, Adventure, Fantasy, Sci-Fi'}]
['Maleficent: Mistress of Evil', 113672999, None, 119, 'PG', datetime.datetime(2019, 10, 18, 0, 0), 'Adventure, Family, Fantasy']


In [37]:
#'https://www.boxofficemojo.com/weekly/by-year/2019/'
#Loop to get Urls based on the example from project 1
def get_urls(years):
    '''Takes a 4 digit year input, or a list of them'''
    url = "https://www.boxofficemojo.com/weekly/by-year/{}/"
    dfs = []
    for year in years:
        file_url = url.format(year)
        response = requests.get(file_url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        
        k = soup.find_all('a')

        df = pd.DataFrame(np.array(list(k)[27:-12]).reshape(-1,3))
        moviepaths = df.loc[:,2]
        #print(pd.DataFrame(moviepaths))
        for item in moviepaths:
            dfs.append(item)
    return dfs
        
years = ['2019','2018','2017','2016','2015','2014','2013','2012','2011','2010']
url_list = get_urls(years)
#url_list



In [34]:
len(url_list)

52

In [ ]:
#url_list

In [40]:
urls = []
for a_tag in url_list:
    url = a_tag['href']
    if not url.startswith('http'):
        url = "https://www.boxofficemojo.com/"+url
    urls.append(url)
len(urls)

522

In [ ]:
'''
test = 'https://www.boxofficemojo.com/weekly/by-year/2019/'
response = requests.get(test)
page = response.text
soup = BeautifulSoup(page, "lxml")

k = soup.find_all('a')
list(k)[27:]

df = pd.DataFrame(np.array(list(k)[27:-12]).reshape(-1,3))
moviepaths = df.loc[:,2]
'''
#urls = [] 
#for a_tag in moviepaths:
#    url = a_tag['href']
#    if not url.startswith('http'):
#        url = "https://www.boxofficemojo.com/"+url
#    urls.append(url)

In [ ]:
'''
dft = pd.DataFrame(columns=Header)

for i in range(52):
    ##info.append(movie_info(urls[i]))
    t = movie_info(urls[i])
    #print(t)
    #Values = list(t[0].values())
    #print(Values)
    #Header = getList(t[0])
    dft = dft.append(t, ignore_index=True)
    
dft
'''

In [41]:
dft = pd.DataFrame(columns=Header)

for i in range(len(urls)):
    ##info.append(movie_info(urls[i]))
    try:
        t = movie_info(urls[i])
    #print(t)
    #Values = list(t[0].values())
    #print(Values)
    #Header = getList(t[0])
        dft = dft.append(t, ignore_index=True)
    except:
        pass
    
dft


,Movie_Title,Domestic_Total_Gross,Budget,Runtime(mins),Rating,Release_Date,Genre
0,Star Wars: The Rise Of Skywalker,482569280,None,142,PG-13,2019-12-20,"Action, Adventure, Fantasy, Sci-Fi"
1,Star Wars: The Rise Of Skywalker,482569280,None,142,PG-13,2019-12-20,"Action, Adventure, Fantasy, Sci-Fi"
2,Jumanji: The Next Level,260185634,"$125,000,000",123,PG-13,2019-12-13,"Action, Adventure, Comedy, Fantasy"
3,Frozen II,460823437,None,103,PG,2019-11-22,"Adventure, Animation, Comedy, Family, Fantasy,..."
4,Frozen II,460823437,None,103,PG,2019-11-22,"Adventure, Animation, Comedy, Family, Fantasy,..."
...,...,...,...,...,...,...,...
503,Avatar,749766139,"$237,000,000",162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi"
504,Avatar,749766139,"$237,000,000",162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi"
505,Avatar,749766139,"$237,000,000",162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi"
506,Avatar,749766139,"$237,000,000",162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi"


In [ ]:
#size(dft)

In [42]:
from datetime import datetime as dt
dft['strpdate'] = pd.to_datetime(dft.Release_Date, format = "%Y-%M-%D")
dft['year'] = pd.DatetimeIndex(dft['strpdate']).year

In [78]:
def Action(item):
    if 'Action' in item:
        return 1
    else:
        return 0
    
def Adventure(item):
    if 'Adventure' in item:
        return 1
    else:
        return 0
    
def Animation(item):
    if 'Animation' in item:
        return 1
    else:
        return 0
    
def Biography(item):
    if 'Biography' in item:
        return 1
    else:
        return 0
    
def Comedy(item):
    if 'Comedy' in item:
        return 1
    else:
        return 0
    
def Crime(item):
    if 'Crime' in item:
        return 1
    else:
        return 0
    
def Documentary(item):
    if 'Documentary' in item:
        return 1
    else:
        return 0
    
def Drama(item):
    if 'Drama' in item:
        return 1
    else:
        return 0
    
def Family(item):
    if 'Family' in item:
        return 1
    else:
        return 0
    
def Fantasy(item):
    if 'Fantasy' in item:
        return 1
    else:
        return 0
    
def History(item):
    if 'History' in item:
        return 1
    else:
        return 0    
    
def Horror(item):
    if 'Horror' in item:
        return 1
    else:
        return 0
    
def Music(item):
    if 'Music' in item:
        return 1
    else:
        return 0
    
def Music(item):
    if 'Music' in item:
        return 1
    else:
        return 0
    
def Musical(item):
    if 'Musical' in item:
        return 1
    else:
        return 0
    
def Mystery(item):
    if 'Mystery' in item:
        return 1
    else:
        return 0
    
def Romance(item):
    if 'Romance' in item:
        return 1
    else:
        return 0
    
def SciFi(item):
    if 'Sci-Fi' in item:
        return 1
    else:
        return 0
    
def Sport(item):
    if 'Sport' in item:
        return 1
    else:
        return 0
    
def Thriller(item):
    if 'Thriller' in item:
        return 1
    else:
        return 0

def War(item):
    if 'War' in item:
        return 1
    else:
        return 0
    
def Western(item):
    if 'Western' in item:
        return 1
    else:
        return 0


In [ ]:
#funclist = [Action,Adventure]#,Animation,Biography]

In [ ]:
#type(funclist[1])

In [ ]:
#dft['Action'] = dft['Genre'].apply(Action)

In [ ]:
#dft.drop(['Music'], axis=1,inplace=True)
#dft.drop(['Crime'], axis=1,inplace=True)
#dft.drop(['Mystery'], axis=1,inplace=True)
#dft

In [44]:
dft.to_pickle('MovieData.pkl')

In [46]:
dfttest = dft.copy()
dfttest.head()

,Movie_Title,Domestic_Total_Gross,Budget,Runtime(mins),Rating,Release_Date,Genre,strpdate,year
0,Star Wars: The Rise Of Skywalker,482569280,None,142,PG-13,2019-12-20,"Action, Adventure, Fantasy, Sci-Fi",2019-12-20,2019
1,Star Wars: The Rise Of Skywalker,482569280,None,142,PG-13,2019-12-20,"Action, Adventure, Fantasy, Sci-Fi",2019-12-20,2019
2,Jumanji: The Next Level,260185634,"$125,000,000",123,PG-13,2019-12-13,"Action, Adventure, Comedy, Fantasy",2019-12-13,2019
3,Frozen II,460823437,None,103,PG,2019-11-22,"Adventure, Animation, Comedy, Family, Fantasy,...",2019-11-22,2019
4,Frozen II,460823437,None,103,PG,2019-11-22,"Adventure, Animation, Comedy, Family, Fantasy,...",2019-11-22,2019


In [67]:
#dfttest['Budget']=dfttest['Budget'].apply(budget_to_int)
dfttest

,Movie_Title,Domestic_Total_Gross,Budget,Runtime(mins),Rating,Release_Date,Genre,strpdate,year
0,Star Wars: The Rise Of Skywalker,482569280,NaN,142,PG-13,2019-12-20,"Action, Adventure, Fantasy, Sci-Fi",2019-12-20,2019
1,Star Wars: The Rise Of Skywalker,482569280,NaN,142,PG-13,2019-12-20,"Action, Adventure, Fantasy, Sci-Fi",2019-12-20,2019
2,Jumanji: The Next Level,260185634,125000000.0,123,PG-13,2019-12-13,"Action, Adventure, Comedy, Fantasy",2019-12-13,2019
3,Frozen II,460823437,NaN,103,PG,2019-11-22,"Adventure, Animation, Comedy, Family, Fantasy,...",2019-11-22,2019
4,Frozen II,460823437,NaN,103,PG,2019-11-22,"Adventure, Animation, Comedy, Family, Fantasy,...",2019-11-22,2019
...,...,...,...,...,...,...,...,...,...
503,Avatar,749766139,237000000.0,162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi",2009-12-18,2009
504,Avatar,749766139,237000000.0,162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi",2009-12-18,2009
505,Avatar,749766139,237000000.0,162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi",2009-12-18,2009
506,Avatar,749766139,237000000.0,162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi",2009-12-18,2009


In [70]:
dfttest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 9 columns):
Movie_Title             508 non-null object
Domestic_Total_Gross    508 non-null object
Budget                  445 non-null float64
Runtime(mins)           498 non-null object
Rating                  504 non-null object
Release_Date            508 non-null datetime64[ns]
Genre                   508 non-null object
strpdate                508 non-null datetime64[ns]
year                    508 non-null int64
dtypes: datetime64[ns](2), float64(1), int64(1), object(5)
memory usage: 35.8+ KB


In [71]:
Glist = list(dft['Genre'].str.split(' ', expand=True).stack().unique())
#Glist[1].strip(',')

In [72]:
Glist2 = []
for item in Glist:
    Glist2.append(item.strip(','))


In [ ]:
#len(Glist2)

In [73]:
Glistset = list(set(Glist2))
len(Glistset)


21

In [81]:
Glistset.sort()
Glistset

['Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Thriller',
 'War',
 'Western']

In [79]:
funclist = [Action, Adventure, Animation, Biography, Comedy, Crime, Documentary, Drama, Family, Fantasy, 
            History, Horror, Music, Musical, Mystery, Romance, SciFi, Sport, Thriller, War, Western]

In [80]:
len(funclist)

21

In [ ]:
#G = ['Action','Adventure']

In [82]:
#for item in G:
#    for func in funclist:
for i in range(len(Glistset)):
        dfttest[Glistset[i]] = dfttest['Genre'].apply(funclist[i])
dfttest
    
    
    
    
    
    

,Movie_Title,Domestic_Total_Gross,Budget,Runtime(mins),Rating,Release_Date,Genre,strpdate,year,Action,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Star Wars: The Rise Of Skywalker,482569280,NaN,142,PG-13,2019-12-20,"Action, Adventure, Fantasy, Sci-Fi",2019-12-20,2019,1,...,0,0,0,0,0,1,0,0,0,0
1,Star Wars: The Rise Of Skywalker,482569280,NaN,142,PG-13,2019-12-20,"Action, Adventure, Fantasy, Sci-Fi",2019-12-20,2019,1,...,0,0,0,0,0,1,0,0,0,0
2,Jumanji: The Next Level,260185634,125000000.0,123,PG-13,2019-12-13,"Action, Adventure, Comedy, Fantasy",2019-12-13,2019,1,...,0,0,0,0,0,0,0,0,0,0
3,Frozen II,460823437,NaN,103,PG,2019-11-22,"Adventure, Animation, Comedy, Family, Fantasy,...",2019-11-22,2019,0,...,0,1,1,0,0,0,0,0,0,0
4,Frozen II,460823437,NaN,103,PG,2019-11-22,"Adventure, Animation, Comedy, Family, Fantasy,...",2019-11-22,2019,0,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,Avatar,749766139,237000000.0,162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi",2009-12-18,2009,1,...,0,0,0,0,0,1,0,0,0,0
504,Avatar,749766139,237000000.0,162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi",2009-12-18,2009,1,...,0,0,0,0,0,1,0,0,0,0
505,Avatar,749766139,237000000.0,162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi",2009-12-18,2009,1,...,0,0,0,0,0,1,0,0,0,0
506,Avatar,749766139,237000000.0,162,PG-13,2009-12-18,"Action, Adventure, Fantasy, Sci-Fi",2009-12-18,2009,1,...,0,0,0,0,0,1,0,0,0,0


In [86]:
dfttest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 30 columns):
Movie_Title             508 non-null object
Domestic_Total_Gross    508 non-null object
Budget                  445 non-null float64
Runtime(mins)           498 non-null object
Rating                  504 non-null object
Release_Date            508 non-null datetime64[ns]
Genre                   508 non-null object
strpdate                508 non-null datetime64[ns]
year                    508 non-null int64
Action                  508 non-null int64
Adventure               508 non-null int64
Animation               508 non-null int64
Biography               508 non-null int64
Comedy                  508 non-null int64
Crime                   508 non-null int64
Documentary             508 non-null int64
Drama                   508 non-null int64
Family                  508 non-null int64
Fantasy                 508 non-null int64
History                 508 non-null int64
Horror    

In [85]:
dfttest.groupby(['year']).Adventure.sum()

year
2009     5
2010    25
2011    25
2012    29
2013    26
2014    31
2015    35
2016    37
2017    31
2018    31
2019    34
Name: Adventure, dtype: int64

In [ ]:
#dft.loc[551,:]


In [ ]:
#t = movie_info(urls[1])
#t

In [ ]:
#Values = list(t[0].values())
#Values

In [ ]:
#Header = getList(t[0])
#Header

In [ ]:
#dft = pd.DataFrame(columns=Header)
#dft.append(pd.Series(Values,index=Header), ignore_index=True)
